In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import textwrap

In [2]:
raw_data = pd.read_excel('./sleep_data.xlsx')
raw_data.rename(columns={'신장':'height', '체중':'weight'}, inplace=True)
raw_data.columns = raw_data.columns.str.replace(' \n', ' ')
raw_data.columns = raw_data.columns.str.replace('\n', ' ')
raw_data.head()

/var/folders/l6/j5nwgjg11zqbq0y11b2myyb00000gn/T/ipykernel_28608/1846465838.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  raw_data = pd.read_excel('./sleep_data.xlsx')


,ID,PSG_date,PSG_ID,sex,age,birth,Final Diagnosis,OSA,insomnia,"PLMD, RLS",...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)
0,1,2014-12-10,N14005,1,52,1962-02-26,sleep talking,0,0,0,...,97.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2014-12-10,PE141015,0,65,1948-12-17,RLS/OSA,1,0,1,...,87.0,0.0,0.0,0.0,16.7,NaN,NaN,NaN,NaN,NaN
2,3,2014-12-10,PE141017,0,53,1961-02-28,OSA,1,0,0,...,87.0,0.0,0.0,0.0,10.7,NaN,NaN,NaN,NaN,NaN
3,4,2014-12-10,PE141014,0,53,1961-06-01,OSA/PPI,1,1,0,...,86.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015-03-31,PE150253,1,55,1959-12-19,OSA,1,0,0,...,88.0,0.0,0.0,0.0,3.2,2.0,0.3,0.0,0.0,0.3


# 데이터 전처리

In [3]:
data = raw_data.copy()
data.drop(['PSG_date', 'PSG_ID', 'birth'], axis=1, inplace=True)
data.head()

,ID,sex,age,Final Diagnosis,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)
0,1,1,52,sleep talking,0,0,0,0,0,0,...,97.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,65,RLS/OSA,1,0,1,0,0,0,...,87.0,0.0,0.0,0.0,16.7,NaN,NaN,NaN,NaN,NaN
2,3,0,53,OSA,1,0,0,0,0,0,...,87.0,0.0,0.0,0.0,10.7,NaN,NaN,NaN,NaN,NaN
3,4,0,53,OSA/PPI,1,1,0,0,0,0,...,86.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,55,OSA,1,0,0,0,0,0,...,88.0,0.0,0.0,0.0,3.2,2.0,0.3,0.0,0.0,0.3


In [4]:
# object type으로 만드는 record 찾기
data[data['Lateral Apnea Index(/h)'].apply(lambda x: isinstance(x, str))]

,ID,sex,age,Final Diagnosis,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)
5076,5077,1,62,OSA,1,0,0,0,0,0,...,81.0,40.8,18.1,0.0,11.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data.drop([5076], inplace=True)
data.reset_index(drop=True, inplace=True)
data[data['Lateral Apnea Index(/h)'].apply(lambda x: isinstance(x, str))]

,ID,sex,age,Final Diagnosis,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)


In [6]:
data['Lateral Apnea Index(/h)'] = data['Lateral Apnea Index(/h)'].astype(float)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5480 entries, 0 to 5479
Data columns (total 87 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ID                                    5480 non-null   int64  
 1   sex                                   5480 non-null   int64  
 2   age                                   5480 non-null   int64  
 3   Final Diagnosis                       5480 non-null   object 
 4   OSA                                   5480 non-null   int64  
 5   insomnia                              5480 non-null   int64  
 6   PLMD, RLS                             5480 non-null   int64  
 7   parasomnia                            5480 non-null   int64  
 8   circadian rhythm sleep-wake disorder  5480 non-null   int64  
 9   Narcolepsy, EDS, Hypersomnia          5480 non-null   int64  
 10  Catathrenia                           5480 non-null   int64  
 11  control          

In [7]:
# 분류된 집단에 속하지 않으며 수면장애 없음으로 분류되지도 않은 record 걸러내기
all_zeros = data.loc[(data['OSA'] == 0) & (data['insomnia'] == 0) & (data['PLMD, RLS'] == 0) & (data['parasomnia'] == 0) & (data['circadian rhythm sleep-wake disorder'] == 0) & (data['Narcolepsy, EDS, Hypersomnia'] == 0) & (data['Catathrenia'] == 0) & (data['control'] == 0)].index
len(all_zeros)

35

In [8]:
# 진단은 OSA만 남겨두고 그 외 전부 drop
osa_data = data.copy()
osa_data.drop(osa_data.iloc[all_zeros, :].index, axis=0, inplace=True)
osa_data.drop(['Final Diagnosis'] + data.iloc[:, 5:12].columns.values.tolist(), axis=1, inplace=True)
osa_data.head()

,ID,sex,age,OSA,height,weight,BMI,Time in Bed(min),Total sleep time(min),Sleep latency(min),...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)
0,1,1,52,0,167.0,60.0,21.51,376.5,276.5,31.0,...,97.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,65,1,166.0,53.0,19.23,513.1,248.0,107.5,...,87.0,0.0,0.0,0.0,16.7,NaN,NaN,NaN,NaN,NaN
2,3,0,53,1,174.0,74.0,24.44,479.0,421.0,8.5,...,87.0,0.0,0.0,0.0,10.7,NaN,NaN,NaN,NaN,NaN
3,4,0,53,1,163.0,63.5,23.90,464.0,343.0,14.0,...,86.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,55,1,150.0,61.0,27.11,464.5,417.0,1.5,...,88.0,0.0,0.0,0.0,3.2,2.0,0.3,0.0,0.0,0.3


In [9]:
numerical_osa_data = osa_data.drop('OSA', axis=1)
numerical_osa_data.head()

,ID,sex,age,height,weight,BMI,Time in Bed(min),Total sleep time(min),Sleep latency(min),N2 sleep latency(min),...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)
0,1,1,52,167.0,60.0,21.51,376.5,276.5,31.0,0.0,...,97.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,65,166.0,53.0,19.23,513.1,248.0,107.5,0.0,...,87.0,0.0,0.0,0.0,16.7,NaN,NaN,NaN,NaN,NaN
2,3,0,53,174.0,74.0,24.44,479.0,421.0,8.5,0.0,...,87.0,0.0,0.0,0.0,10.7,NaN,NaN,NaN,NaN,NaN
3,4,0,53,163.0,63.5,23.90,464.0,343.0,14.0,0.0,...,86.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,55,150.0,61.0,27.11,464.5,417.0,1.5,1.5,...,88.0,0.0,0.0,0.0,3.2,2.0,0.3,0.0,0.0,0.3


In [10]:
numerical_osa_data.describe()

,ID,sex,age,height,weight,BMI,Time in Bed(min),Total sleep time(min),Sleep latency(min),N2 sleep latency(min),...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)
count,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5444.00000,...,5410.00000,5445.000000,5445.000000,5445.000000,5315.000000,5244.000000,5244.000000,5243.000000,5243.000000,5244.000000
mean,2742.092929,0.289073,50.860239,168.157062,73.350156,27.512320,431.502571,355.405418,12.150725,7.41982,...,84.50037,17.963067,6.478880,0.018916,10.508881,1.709001,0.294308,3.953843,0.696672,0.996777
std,1579.758776,0.453373,14.873225,26.695336,38.480032,43.906569,52.727879,66.721425,19.030153,14.05947,...,8.15235,21.868456,15.510489,0.861512,21.063967,4.077729,0.703895,12.598826,2.228601,2.597956
min,1.000000,0.000000,7.000000,100.000000,30.000000,12.400000,96.900000,46.500000,0.000000,0.00000,...,31.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1376.000000,0.000000,40.000000,162.000000,62.000000,22.940000,403.000000,318.000000,3.000000,1.00000,...,81.00000,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2745.000000,0.000000,53.000000,168.000000,71.000000,25.100000,436.000000,362.000000,6.500000,3.50000,...,86.00000,9.400000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4107.000000,1.000000,62.000000,174.000000,80.000000,27.680000,467.000000,402.500000,13.000000,7.50000,...,90.00000,27.700000,4.600000,0.000000,12.400000,2.000000,0.300000,1.000000,0.200000,0.700000
max,5481.000000,1.000000,91.000000,999.000000,999.000000,999.000000,642.000000,537.500000,303.000000,224.00000,...,99.00000,141.900000,169.400000,44.500000,187.300000,129.000000,21.500000,218.000000,37.500000,38.800000


In [11]:
print(osa_data.shape)
print(osa_data[osa_data['OSA'] == 0].shape)
print(osa_data[osa_data['OSA'] == 1].shape)

(5445, 79)
(1349, 79)
(4095, 79)


In [12]:
# OSA 이상치 확인
osa_data.loc[(osa_data['OSA'] != 0) & (osa_data['OSA'] != 1)]

,ID,sex,age,OSA,height,weight,BMI,Time in Bed(min),Total sleep time(min),Sleep latency(min),...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)
3570,3571,0,52,999,999.0,999.0,999.0,408.1,359.0,8.6,...,78.0,23.4,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
osa_data.drop(osa_data.loc[(osa_data['OSA'] != 0) & (osa_data['OSA'] != 1)].index, axis=0, inplace=True)
osa_data.loc[(osa_data['OSA'] != 0) & (osa_data['OSA'] != 1)]

,ID,sex,age,OSA,height,weight,BMI,Time in Bed(min),Total sleep time(min),Sleep latency(min),...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)


In [14]:
# 신체 정보 이상치 확인
osa_na_data = osa_data.loc[(osa_data['sex'] == 999) | (osa_data['age'] == 999) | (osa_data['height'] == 999) | (osa_data['weight'] == 999) | (osa_data['BMI'] == 999)]
osa_na_data

,ID,sex,age,OSA,height,weight,BMI,Time in Bed(min),Total sleep time(min),Sleep latency(min),...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)
412,413,1,65,1,100.0,999.0,999.0,378.5,307.5,4.0,...,86.0,11.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1171,1172,0,53,1,173.0,54.0,999.0,429.5,351.5,11.0,...,89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1508,1509,1,55,0,162.0,999.0,999.0,470.5,252.5,42.5,...,92.0,1.0,0.0,0.0,22.3,4.0,1.0,3.0,0.7,1.7
1945,1946,0,55,1,999.0,999.0,999.0,356.0,328.0,1.0,...,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2414,2415,0,69,1,999.0,999.0,999.0,455.0,298.0,5.5,...,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2891,2892,0,68,1,161.0,59.0,999.0,476.5,371.0,21.1,...,84.0,21.5,2.5,0.0,23.0,7.0,1.1,11.0,1.8,2.9
3977,3978,0,21,1,999.0,999.0,999.0,485.5,402.0,2.0,...,69.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4124,4125,0,68,1,999.0,999.0,999.0,426.0,330.0,1.0,...,80.0,24.7,10.7,0.0,18.0,8.0,1.5,14.0,2.5,4.0
4896,4897,0,48,1,166.0,999.0,999.0,369.0,340.0,5.5,...,86.0,12.5,0.2,0.0,4.1,2.0,0.4,4.0,0.7,1.1
4913,4914,0,64,1,164.0,67.0,999.0,476.0,427.5,3.5,...,72.0,0.0,0.0,0.0,32.3,NaN,NaN,NaN,NaN,NaN


In [15]:
del numerical_osa_data
osa_data.drop(osa_na_data.index, inplace=True)

numerical_osa_data = osa_data.drop('OSA', axis=1)
numerical_osa_data.describe()

,ID,sex,age,height,weight,BMI,Time in Bed(min),Total sleep time(min),Sleep latency(min),N2 sleep latency(min),...,Lowest SaO2 (%),ODI,90%ODI,EtCO2 (>50mmHg),Total LMI =PLMI(/h),Total LM Arousal#,Total LM AI(/h),PLM Arousal#,PLM AI(/h),MAI(/h)
count,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5433.000000,...,5399.000000,5434.000000,5434.000000,5434.000000,5304.000000,5234.000000,5234.000000,5233.000000,5233.000000,5234.000000
mean,2741.785793,0.289290,50.849466,167.407840,71.994774,25.545745,431.505502,355.431634,12.155870,7.429137,...,84.506946,17.982057,6.487413,0.018955,10.511878,1.708254,0.294106,3.955284,0.696914,0.996828
std,1579.979557,0.453474,14.874485,8.795353,14.835601,4.119163,52.742871,66.753848,19.041692,14.070709,...,8.151987,21.881901,15.523981,0.862383,21.078961,4.079506,0.704143,12.609001,2.230413,2.599735
min,1.000000,0.000000,7.000000,123.000000,30.000000,12.400000,96.900000,46.500000,0.000000,0.000000,...,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1375.250000,0.000000,40.000000,162.000000,62.000000,22.910000,403.000000,318.000000,3.000000,1.000000,...,81.000000,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2744.500000,0.000000,53.000000,168.000000,71.000000,25.100000,436.000000,362.000000,6.500000,3.500000,...,86.000000,9.400000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4106.750000,1.000000,62.000000,174.000000,80.000000,27.600000,467.000000,402.500000,13.000000,7.500000,...,90.000000,27.700000,4.600000,0.000000,12.400000,2.000000,0.300000,1.000000,0.200000,0.700000
max,5481.000000,1.000000,91.000000,198.000000,181.000000,57.020000,642.000000,537.500000,303.000000,224.000000,...,99.000000,141.900000,169.400000,44.500000,187.300000,129.000000,21.500000,218.000000,37.500000,38.800000


In [16]:
# 결측치 확인
na_columns = osa_data.isna().sum()
na_columns = na_columns[na_columns > 0]
na_columns

N2 sleep latency(min)      1
WASO(min)                  1
WASO(%)                    1
Spontaneous Arousal #     34
Movement Arousal #         8
Total_ apnea #             3
Total_ Hypopnea#           3
Baseline waking SaO2      22
Baseline sleep SaO2       22
Lowest SaO2 (%)           35
Total LMI =PLMI(/h)      130
Total LM Arousal#        200
Total LM AI(/h)          200
PLM Arousal#             201
PLM AI(/h)               201
MAI(/h)                  200
dtype: int64

## A. 결측치를 포함하는 record drop

In [17]:
osa_data_a = osa_data.dropna(axis=0, how="any")

In [18]:
del na_columns

na_columns = osa_data_a.isna().sum()
na_columns = na_columns[na_columns > 0]
na_columns

Series([], dtype: int64)

In [19]:
osa_data_a.shape

(5152, 79)

## B. 결측치를 포함하는 feature drop

In [20]:
osa_data_b = osa_data.dropna(axis=1, how="any")

In [21]:
del na_columns

na_columns = osa_data_b.isna().sum()
na_columns = na_columns[na_columns > 0]
na_columns

Series([], dtype: int64)

In [22]:
osa_data_b.shape

(5434, 63)

# feature engineering 및 모델별 성능 확인

### 평가 지표 함수

In [23]:
from sklearn.metrics import *

def eval_metrics(y_test, pred=None, pred_proba=None):
    # cm = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('accuracy: {0:.4f}, precision: {1:.4f}, recall: {2:.4f}, f1: {3:.4f}, roc_auc: {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

## 1. 별다른 feature engineering 없이 진행

## 1-1. Logistic Regression

### A. with osa_data_a

In [24]:
from sklearn.model_selection import train_test_split

data = osa_data_a.drop(['OSA'], axis=1)
target = osa_data_a['OSA']

# 불균형으로 인해 stratify 불가
x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(data, target, test_size=0.2, random_state=42)

In [25]:
print(x_train_a.shape)
print(x_test_a.shape)

(4121, 78)
(1031, 78)


In [26]:
from sklearn.preprocessing import MinMaxScaler

features = list(x_train_a.columns)
train = x_train_a.copy()
test = x_test_a.copy()

scaler = MinMaxScaler(feature_range=(0,1))

scaler.fit(train)
train_a = scaler.transform(train)
test_a = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (4121, 78)
Testing data shape:  (1031, 78)


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_dict = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
    'penalty': ['l1', 'l2']
}

gs_log_reg = GridSearchCV(LogisticRegression(max_iter=700, solver='liblinear'), param_dict, cv=3, refit=True)
gs_log_reg.fit(train_a, y_train_a)

print(gs_log_reg.best_params_)
print(gs_log_reg.best_score_)

{'C': 1.0, 'penalty': 'l1'}
0.908274856498076


In [29]:
y_pred_proba = gs_log_reg.best_estimator_.predict_proba(test_a)[:,1]
y_pred = (y_pred_proba > 0.5).astype(int)
y_pred

array([1, 1, 1, ..., 0, 1, 1])

In [30]:
eval_metrics(y_test_a, pred=y_pred, pred_proba=y_pred_proba)

accuracy: 0.9195, precision: 0.9515, recall: 0.9404, f1: 0.9459, roc_auc: 0.9656


### B. with osa_data_b

In [31]:
data = osa_data_b.drop(['OSA'], axis=1)
target = osa_data_b['OSA']

# 불균형으로 인해 stratify 불가
x_train_b, x_test_b, y_train_b, y_test_b = train_test_split(data, target, test_size=0.2, random_state=42)

In [32]:
print(x_train_b.shape)
print(x_test_b.shape)

(4347, 62)
(1087, 62)


In [33]:
features = list(x_train_b.columns)
train = x_train_b.copy()
test = x_test_b.copy()

scaler = MinMaxScaler(feature_range=(0,1))

scaler.fit(train)
train_b = scaler.transform(train)
test_b = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (4347, 62)
Testing data shape:  (1087, 62)


In [34]:
gs_log_reg = GridSearchCV(LogisticRegression(max_iter=700, solver='liblinear'), param_dict, cv=3, refit=True)
gs_log_reg.fit(train_b, y_train_b)

print(gs_log_reg.best_params_)
print(gs_log_reg.best_score_)

{'C': 1.0, 'penalty': 'l1'}
0.9061421670117321


In [35]:
y_pred_proba = gs_log_reg.best_estimator_.predict_proba(test_b)[:,1]
y_pred = (y_pred_proba > 0.5).astype(int)
y_pred

array([0, 1, 1, ..., 1, 1, 1])

In [36]:
eval_metrics(y_test_b, pred=y_pred, pred_proba=y_pred_proba)

accuracy: 0.9328, precision: 0.9589, recall: 0.9531, f1: 0.9560, roc_auc: 0.9744


#### Logistic Regression
- A(결측치 record drop)
    - train: 0.908274856498076
    - test: 0.9194956353055286
- B(결측치 feature drop)
    - train: 0.9066022544283414
    - test: 0.9310027598896045

## 1-2. SVM

### A. with osa_data_a

In [37]:
from sklearn.svm import SVC

params_dict = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'gamma': [0.001, 0.01, 0.1, 1.0, 10.0]
}

grid_svc = GridSearchCV(SVC(probability=True), params_dict, cv=3, refit=True)
grid_svc.fit(train_a, y_train_a)

print(grid_svc.best_params_)
print(grid_svc.best_score_)

{'C': 1.0, 'gamma': 1.0}
0.9090024818420548


In [40]:
y_pred_proba = grid_svc.best_estimator_.predict_proba(test_a)[:,1]
y_pred = (y_pred_proba > 0.5).astype(int)
y_pred

array([1, 1, 1, ..., 0, 1, 1])

In [41]:
eval_metrics(y_test_a, pred=y_pred, pred_proba=y_pred_proba)

accuracy: 0.9088, precision: 0.9346, recall: 0.9443, f1: 0.9394, roc_auc: 0.9483


### B. with osa_data_b

In [42]:
grid_svc = GridSearchCV(SVC(probability=True), params_dict, cv=3, refit=True)
grid_svc.fit(train_b, y_train_b)

print(grid_svc.best_params_)
print(grid_svc.best_score_)

{'C': 10.0, 'gamma': 0.1}
0.9118932597193466


In [43]:
y_pred_proba = grid_svc.best_estimator_.predict_proba(test_b)[:,1]
y_pred = (y_pred_proba > 0.5).astype(int)
y_pred

array([0, 1, 1, ..., 1, 1, 1])

In [44]:
eval_metrics(y_test_b, pred=y_pred, pred_proba=y_pred_proba)

accuracy: 0.9236, precision: 0.9550, recall: 0.9447, f1: 0.9498, roc_auc: 0.9738


#### SVM
- A(결측치 record drop)
    - train: 0.9090024818420548
    - test: 0.9088263821532493
- B(결측치 feature drop)
    - train: 0.9118932597193466
    - test: 0.922723091076357

## 1-3. Random Forest

### A. with osa_data_a

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

params_dict = {
    'n_estimators': [10, 50, 100],
    'max_depth': [2, 4, 6, 8, 10, 12],
    'min_samples_leaf': [2, 4, 6, 8, 12, 18],
    'min_samples_split': [2, 4, 6, 8, 16, 20]
}

grid_rf = GridSearchCV(RandomForestClassifier(), params_dict, cv=3, refit=True)
grid_rf.fit(train_a, y_train_a)
# random_rf = RandomizedSearchCV(RandomForestClassifier(), params_dict, n_iter=50, cv=3, verbose=1)
# random_rf.fit(train_a, y_train_a)

print(grid_rf.best_params_)
print(grid_rf.best_score_)

{'max_depth': 8, 'min_samples_leaf': 12, 'min_samples_split': 4, 'n_estimators': 10}
0.9160394917860323


In [54]:
y_pred_proba = grid_rf.best_estimator_.predict_proba(test_a)[:,1]
y_pred = (y_pred_proba > 0.5).astype(int)
y_pred

array([1, 1, 1, ..., 0, 1, 1])

In [55]:
eval_metrics(y_test_a, pred=y_pred, pred_proba=y_pred_proba)

accuracy: 0.9340, precision: 0.9411, recall: 0.9728, f1: 0.9567, roc_auc: 0.9624


### B. with osa_data_b

In [58]:
grid_rf = GridSearchCV(RandomForestClassifier(), params_dict, cv=3, refit=True)
grid_rf.fit(train_b, y_train_b)

print(grid_rf.best_params_)
print(grid_rf.best_score_)

{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
0.9164941338854383


In [59]:
y_pred_proba = grid_rf.best_estimator_.predict_proba(test_b)[:,1]
y_pred = (y_pred_proba > 0.5).astype(int)
y_pred

array([0, 1, 1, ..., 1, 1, 1])

In [60]:
eval_metrics(y_test_b, pred=y_pred, pred_proba=y_pred_proba)

accuracy: 0.9273, precision: 0.9373, recall: 0.9700, f1: 0.9533, roc_auc: 0.9749
